In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
from pathlib import Path
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({
    "pgf.texsystem": "xelatex",
    'font.family': 'serif',
    'text.usetex': False,
    'pgf.rcfonts': False,
    'figure.dpi': 300,
})
import seaborn as sns

In [39]:
exp_mtx = pd.read_csv(r'g:\spatial_data\processed\20230705_PRISM3D_mousebrain_HT_confocal\segmented\expression_matrix.csv', index_col=0)
gene_list = [_.upper() for _ in exp_mtx.columns]

In [10]:
mapped_genes_raw = pd.read_csv('detected_transcripts_S2R1.csv', index_col=0)

In [40]:
mapped_genes = mapped_genes_raw[mapped_genes_raw['gene'].map(lambda x: x.upper()).isin(gene_list)]
print(mapped_genes['gene'].unique())

['Gad1' 'Slc17a7' 'Gfap' 'Cx3cr1' 'Aqp4']


In [41]:
mapped_genes.to_csv('detected_transcripts_S2R1_overlap.csv')
mapped_genes.head()

,barcode_id,global_x,global_y,global_z,x,y,fov,gene
5,379,64.45234,3482.4714,0.0,176.20090,139.73760,1,Gad1
32,379,299.31070,3092.4124,0.0,419.14548,227.97830,9,Gad1
54,384,266.33380,3595.0205,0.0,195.38359,1155.00680,11,Slc17a7
55,384,275.34300,3597.0708,1.0,278.80200,1173.99010,11,Slc17a7
74,420,241.69711,4290.3150,0.0,144.36629,154.38948,15,Gfap


In [42]:
mapped_genes['Y'] = mapped_genes['global_y']
mapped_genes['X'] = mapped_genes['global_x']

In [43]:
from glob import glob
from tifffile import imwrite

density_dir = Path('density')
density_dir.mkdir(exist_ok=True)


im_shape = int(mapped_genes['Y'].max())+2, int(mapped_genes['X'].max())+2
print(im_shape)

def plot_density_downsample(df, fac=20):
    y = (im_shape[0] // fac) + 1
    x = (im_shape[1] // fac) + 1
    coordinates = df[['Y','X']].to_numpy()
    canvas = np.zeros((y*fac, x*fac),dtype=np.uint16)
    canvas[coordinates[:,0], coordinates[:,1]] = 1
    canvas_down = canvas.reshape(y,fac,x,fac).sum(-1).sum(1)
    return canvas_down

for gene in tqdm(mapped_genes['gene'].unique()):
    # if glob(str(density_dir / f'{gene}.tif')): continue
    # else: 
    image = plot_density_downsample(mapped_genes[mapped_genes['gene']==gene][['Y', 'X']].astype(int))
    imwrite(density_dir/f'{gene}.tif', image.astype(np.uint16))

(7454, 9495)


100%|██████████| 5/5 [00:01<00:00,  2.75it/s]
